In [17]:
include("./src/parse_file.jl")

Completed analyis of 63 reads in chr1
Completed analyis of 37 reads in chr2


Dict{String, SparseMatrixCSC{Int64, Int64}} with 2 entries:
  "chr2" => sparse([2, 2, 4, 6, 2, 4, 6, 4, 6, 4  …  15, 15, 16, 16, 17, 18, 17…
  "chr1" => sparse([2, 2, 2, 2, 2, 3, 3, 1, 1, 4  …  23, 24, 25, 26, 27, 21, 22…

In [18]:
path = "./testing/test_epiread_small.bed"
big_bgz_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/test_epiread.bed.bgz"
part_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"


"/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"

In [19]:
results = analyze_file(path)

Completed analyis of 63 reads in chr1
Completed analyis of 37 reads in chr2


Dict{String, SparseMatrixCSC{Int64, Int64}} with 2 entries:
  "chr2" => sparse([2, 2, 4, 6, 2, 4, 6, 4, 6, 4  …  15, 15, 16, 16, 17, 18, 17…
  "chr1" => sparse([2, 2, 2, 2, 2, 3, 3, 1, 1, 4  …  23, 24, 25, 26, 27, 21, 22…

In [160]:
size(results["chr1"], 2)

631931

In [37]:
test = x[:,136617:136620]

27×4 SparseMatrixCSC{Int64, Int64} with 7 stored entries:
⢨⢨
⠀⠈
⠀⠀
⠀⠀
⠀⠀
⠀⠀
⠀⠀